# nPE: A Configurable Processing Engine
#### Documentation | Version: 0.4.1 | Updated 2018.7.25
---

## Introduction

The nPE is a highly configurable processing engine that can be topologically configured to support various DNN accelerator architectures. Its functionality is able to support inner products with varying amounts of spatial and temporal parallelism, nonlinear activation functions, scalar addition, scalar multiplication, and max functions. It supports many existing architectures and dataflows, such as Eyeriss, Flexflow, and ShiDianNao. Its generality based on the notion of soft, firm, and hard bypasses.
- Soft bypasses make it possible to bypass or not bypass via control signals and a multiplexer.
- Firm bypasses add a bypass line as an output.
- Hard bypasses hardwire the input and output together.

The details of bypass behavior change depending on the specific module.

It's microarchitecture consists of two multiport Register Files, an n-dimensional inner product unit, an RF-with-ALU combo, and a nonlinear processing unit. All three registers are accessible from an external NoC, and all arithmetic modules support an "Identity" operation.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = """
C:\Users\RyanL\OneDrive\Research\SEAL\processing-engine/source/load-ivy.sc
"""

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.math.pow

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}


import scala.math.pow

## Parallel Register File

The Parallel Register File (pRF) is a register file that is configurable based on number of ports in/out, register size, bypass type, and bitwidth. Bypass options are None, Soft, or Hard. The bypass behavior is such that it bypasses in[k] to out[k], where k is the port index. Without loss of generality, the bypass option will be applied to all ports. Multiple bypass options are enabled by parallel pRFs.

### Single Register File

The Single Register File acts as expected: it reads data in when write enable is high and reads data when read enable is high. It can read and write in parallel. It has two read ports: one for internal PE reading and one for external PE communication. The register size adjusts based on the bitwidth of the addresses.

In [3]:
class RFConfig(
        val addrWidth: Int, 
        val dataWidth: Int,
        val numInputs: Int,
        val bpSupport: Boolean)

class RFControl(c: RFConfig) extends Bundle {
    
    override def cloneType = (new RFControl(c)).asInstanceOf[this.type]
    
    val wEnable = Bool()
    val rEnable = Bool()
    val wAddr = UInt(c.addrWidth.W)
    val rAddrInt = UInt(c.addrWidth.W)
    val rAddrExt = UInt(c.addrWidth.W)
    val bpSel = if (c.bpSupport) Some(Bool()) else None
    val inSel = if (c.numInputs > 1) Some(Vec(c.numInputs, Bool())) else None
}

class RFInput(c: RFConfig) extends Bundle {
    
    override def cloneType = (new RFInput(c)).asInstanceOf[this.type] 
    
    val data = Vec(c.numInputs, SInt(c.dataWidth.W))
}

class RFOutput(c: RFConfig) extends Bundle {
    
    override def cloneType = (new RFOutput(c)).asInstanceOf[this.type]
    
    val int = SInt(c.dataWidth.W)
    val ext = SInt(c.dataWidth.W)
}

class RF(c: RFConfig) extends Module {
    
    val io = IO(new Bundle {
        val control = Input(new RFControl(c))
        val in = Input(new RFInput(c))
        val out = Output(new RFOutput(c))
    })
    
    val zeros = Vec(Seq.fill(pow(2, c.addrWidth).toInt) { 0.S(c.dataWidth.W) })
    val registers = RegInit(zeros)
    
    val sel = io.control.inSel.getOrElse(Vec(true.B))
    
    when (io.control.wEnable) {
        registers(io.control.wAddr) := PriorityMux(sel, io.in.data)
    }
    
    when (io.control.rEnable) {
        when(io.control.bpSel.getOrElse(false.B)) {
            io.out.int := PriorityMux(sel, io.in.data)
            io.out.ext := PriorityMux(sel, io.in.data)
        } .otherwise {
            io.out.int := registers(io.control.rAddrInt)
            io.out.ext := registers(io.control.rAddrExt)
        }
    } .otherwise {
        io.out.int := 0.S
        io.out.ext := 0.S
    }
}

defined class RFInputs
defined class RFOutputs
defined class RF

### Putting them Together

The single RFs come together in parallel to form the pRF. The number of ports is the number of single RFs. Thus, there are all of the control signals associated with a single RF, but with a bus width equal to the port count. If the bypass type is Soft, then each RF has an independent bypass control signal.

In [4]:
class PRFConfig(
        val ports: Int,
        val dataWidth: Int,
        val addrWidth: Int,
        val numInputs: Int,
        val bpType: String) {
    
    require(List("None", "Soft", "Hard") contains bpType)
    
    val bpNone = (bpType == "None")
    val bpSoft = (bpType == "Soft")
    val bpHard = (bpType == "Hard")
    
    val rfConfig = new RFConfig(addrWidth, dataWidth, numInputs, bpSoft)
}

class PRFControl(c: PRFConfig) extends Bundle {
    
    override def cloneType = (new PRFControl(c)).asInstanceOf[this.type]
    
    val rf = Vec(c.ports, new RFControl(c.rfConfig))
}

class PRFInput(c: PRFConfig) extends Bundle {
    
    override def cloneType = (new PRFInput(c)).asInstanceOf[this.type]
    
    val rf = Vec(c.ports, new RFInput(c.rfConfig))
}

class PRFOutput(c: PRFConfig) extends Bundle {
    
    override def cloneType = (new PRFOutput(c)).asInstanceOf[this.type]
    
    val rf = Vec(c.ports, new RFOutput(c.rfConfig))
}

class PRF(c: PRFConfig) extends Module {
    
    val io = IO(new Bundle {
        val control = Input(new PRFControl(c))
        val in = Input(new PRFInput(c))
        val out = Output(new PRFOutput(c))
    })
    
    if (c.bpNone || c.bpSoft) {
        
        val rf = Seq.fill(c.ports){ Module(new RF(c.rfConfig)) }
        
        rf.zipWithIndex.map {
            case (x: RF, i: Int) => { 
                x.io.control <> io.control.rf(i)
                x.io.in <> io.in.rf(i)
                io.out.rf(i) <> x.io.out
            } 
        }
        
    } else if (c.bpHard) {
        for (i <- 0 until c.ports) {
            io.out.rf(i).int := io.in.rf(i).data
            io.out.rf(i).ext := io.in.rf(i).data
        }
    }
}

defined class PRFInputs
defined class PRFOutputs
defined class PRF

## Inner Product Unit

The Inner Product Unit (IPU) performs an inner product between two vectors of a configurable length. For SIMD support, this should be configured to be wide. For traditional PEs, such as Eyeriss, note that setting the width to 1 is equivalent to a scalar multiplication.

Bypass options are None or Firm. For Firm, there only one pair of weights and activations may be bypassed at a time, such that the IPU may interface with only one ALU. Thus if the bypass type is Firm, then there is a bypass selection signal that selects which input to feed to the output. This may be generalized to interface with multiple TUs, but not at this time.

There are plans to implement both Soft and Hard bypassing. There are also plans to make using FMA modules an option.

The IPU contains two embedded modules: a parallel multiplier and a reduction tree.

### Parallel Multiplier

The parallel multiplier simply takes two vector inputs of width n and outputs the element-wise multiplication. There are plans to make the multiplier type configurable, *e.g.* serial, combinational, pipelined *etc*.

In [5]:
class PMultConfiguration(val numPairs: Int, val bitWidth: Int) {
    require(numPairs >= 1, "Must have at least one pair of multiplicands.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
}

class PMultInput(numPairs: Int, bitWidth: Int) extends Bundle {
    
    override def cloneType = (new PMultInput(numPairs, bitWidth)).asInstanceOf[this.type]
    
    val weight = Vec(numPairs, SInt(bitWidth.W))
    val actvtn = Vec(numPairs, SInt(bitWidth.W))
}

class PMult(config: PMultConfiguration) extends Module {
    
    val np = config.numPairs
    val bw = config.bitWidth
    
    val io = IO(new Bundle {
        val in = Input(new PMultInput(np, bw))
        val prod = Output(Vec(np, SInt(bw.W)))
    })
    
    io.prod := (io.in.weight zip io.in.actvtn).map { case(a, b) => a * b }
}

defined function checkParamsPMult
defined class PMultInputs
defined class PMult

### Additive Reduction Tree

The additive reduction tree outputs the L1 norm of the n-dim input vector using log(n) layers of 2-input adders. In other words, it sums the input.

There are plans to pipeline this (as well as everything else...).

In [6]:
class AdditiveRTConfiguration(val numAddends: Int, val bitWidth: Int) {
    require(numAddends >= 1, "Number of addends must be at least one.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
}

// Recursively creates a balanced syntax tree
def adjReduce[A](xs: List[A], op: (A, A) => A): A = xs match {
    case Nil => throw new IllegalArgumentException
    case List(single) => single
    case default => {
        val grouped = default.grouped(2).toList
        val result = for (g <- grouped) yield {
            g match {
                case List(a, b) => op(a, b)
                case List(x) => x
            }
        }
        adjReduce(result, op)
    }
}

class AdditiveRT(config: AdditiveRTConfiguration) extends Module {

    val na = config.numAddends
    val bw = config.bitWidth
    
    val io = IO(new Bundle {
        val in  = Input(Vec(na, SInt(bw.W)))
        val sum = Output(SInt(bw.W))
    })
    
    io.sum := adjReduce(io.in toList, (x: SInt, y: SInt) => x + y)
}

defined function nonassocPairwiseReduce
defined function checkParamsAdditiveRT
defined class AdditiveRT

### Putting them Together

Together, the parallel multiplier and additive reduction tree perform an n-dim inner product. They are simply connected output to input. During use of the PE, the bypass type of the IPU is dictated by the functionality of the ALU.

In [7]:
class IPUConfig(val width: Int, val bitWidth: Int, val bpType: String) {
    
    private val bypssError = "Bypass must be \"None\" or \"Firm\""
    private val widthError = "Width must be at least one"
    private val bitWdError = "Data bitwidth must be non-negative"
    
    val supportedBp = List("None", "Firm")
    
    require(width >= 1, widthError)
    require(supportedBp.contains(bpType), bypssError)
    require(bitWidth >= 0, bitWdError)
    
    val childPMultConfig = new PMultConfig(width, bitWidth)
    val childARTreeConfig = new ARTreeConfig(width, bitWidth)
    
    val bpFirm = (bpType == "Firm")
}

class IPUOutput(c: IPUConfig) extends Bundle {
    
    override def cloneType = (new IPUOutput(c)).asInstanceOf[this.type]
    
    val innerProd = Output(SInt(c.bitWidth.W))
    val bpWeight = if (c.bpFirm) Some(SInt(c.bitWidth.W)) else None
    val bpActvtn = if (c.bpFirm) Some(SInt(c.bitWidth.W)) else None
}


class IPU(c: IPUConfig) extends Module {
    
    val cPMConfig = c.childPMultConfig
    val cARTConfig = c.childARTreeConfig
    
    val io = IO(new Bundle {
        val dataIn = Input(new PMultInput(cPMConfig))
        val dataOut = Output(new IPUOutput(c))
        val bpSel = if (c.bpFirm) Some(Input(Vec(c.width, Bool()))) else None
    })
    
    val pMult = Module(new PMult(cPMConfig))
    pMult.io.in <> io.dataIn
    
    val aRTree = Module(new ARTree(cARTConfig))
    aRTree.io.in := pMult.io.prod
    
    io.dataOut.innerProd := aRTree.io.sum
    
    if (c.bpFirm) {
        io.dataOut.bpWeight.get := PriorityMux(io.bpSel.get, io.dataIn.weight)
        io.dataOut.bpActvtn.get := PriorityMux(io.bpSel.get, io.dataIn.actvtn)
    }
}

defined function checkParamsIPU
defined class IPUInputs
defined class IPUOutputs
defined class IPU

## ALU

The ALU is configurable based on the functions it should support and the data bitwidth. The minimum ALU simply connects the input and the output directly. Accumulate means add the local register file output to the inner product from the IPU. Add and Max do perform their respective operation on the two bypasses from the IPU.

Selecting the operation to perfom is done with a one-hot select signal. The order is always Identity, Add, Max, then Accumulate, but the exact indices will change depending on which are supported.

There are plans to add bypass behavior. Right now, it is supported with an identity operation.

In [8]:
class ALUConfig(val dataWidth: Int, val funcs: List[String]) {
    val identityError = "ALU functions must explicitly include Identity."
    val functionError = "Unsupported Error"
    val supportedFuncs = List("Identity", "Add", "Max", "Accumulate")
    
    require(funcs.contains("Identity"), identityError)
    for(x <- funcs) { require(supportedFuncs.contains(x), functionError) }
    
    val addSupp = funcs.contains("Add")
    val maxSupp = funcs.contains("Max")
    val accSupp = funcs.contains("Accumulate")
    val addBypassIn = addSupp || maxSupp
    val numFuncs = funcs.length
}

class ALUInput(c: ALUConfig) extends Bundle {
    
    override def cloneType = (new ALUInput(c)).asInstanceOf[this.type]
    
    val innerProd = Input(SInt(c.dataWidth.W))
    val funcSel = Input(Vec(c.numFuncs, Bool()))
    
    val weightBp = if(c.addBypassIn) Some(Input(SInt(c.dataWidth.W))) else None
    val actvtnBp = if(c.addBypassIn) Some(Input(SInt(c.dataWidth.W))) else None
    val rfFeedback = if(c.accSupp) Some(Input(SInt(c.dataWidth.W))) else None
}

class ALU(c: ALUConfig) extends Module {
 
    val io = IO(new Bundle {
        val in = new ALUInput(c)
        val out = Output(SInt(c.dataWidth.W))
    })
    
    val idnOut = Some(Wire(SInt(c.dataWidth.W)))
    val addOut = if(c.addSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    val maxOut = if(c.maxSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    val accOut = if(c.accSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    
    idnOut.get := io.in.innerProd
    
    if (c.addSupp) { addOut.get := io.in.weightBp.get + io.in.actvtnBp.get }
    if (c.accSupp) { accOut.get := io.in.innerProd + io.in.rfFeedback.get }
    if (c.maxSupp) {
        when (io.in.weightBp.get > io.in.actvtnBp.get) {
            maxOut.get := io.in.weightBp.get
        } .otherwise {
            maxOut.get := io.in.actvtnBp.get
        }
    }
    
    val inters = (idnOut :: addOut :: maxOut :: accOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.out := PriorityMux(io.in.funcSel, inters)
}

defined function checkparamsALU
defined class ALUInputs
defined class ALU

## Feedback Register File

The feedback register file is just a pRF of size 1 as implemented earlier. Its bypass behavior is key to implementing architectures such as FlexFlow. A Hard bypass will connect the input from the NoC to the NLU/ALU and the input from the ALU to the NoC.

## Nonlinear Unit

The Nonlinear Unit (NLU) performs the nonlinear activations functions present in neural networks. As of right now, it supports the Identity function and ReLu, but there are plans to implement sigmoid and tanh using LUTs and linear interpolation.

In [9]:
class NLUConfig(val dataWidth: Int, val funcs: List[String]) {
    
    val supportedFuncs = List("Identity", "ReLu")
    val identityError = "NLU functions must explicitly include Identity."
    val functionError = "Unsupported Function"
    
    require(funcs.contains("Identity"), identityError)
    for(x <- funcs)(require(supportedFuncs.contains(x), functionError))
    
    val reluSupp = funcs.contains("ReLu")
    val numFuncs = funcs.length
}

class NLUInputs(c: NLUConfig) extends Bundle {
    
    override def cloneType = (new NLUInputs(c)).asInstanceOf[this.type]
    
    val data = SInt(c.dataWidth.W)
    val fSel = Vec(c.numFuncs, Bool())
}

class NLU(c: NLUConfig) extends Module {
    
    val io = IO(new Bundle {
        val in  = Input(new NLUInputs(c))
        val out = Output(SInt(c.dataWidth.W))
    })
    
    val idRes   = Some(Wire(SInt(c.dataWidth.W)))
    val reluRes = if(c.reluSupp) Some(Wire(SInt(c.dataWidth.W))) else None
    
    idRes.get := io.in.data
    
    if (c.reluSupp) {
        when (io.in.data > 0.S) {
            reluRes.get := io.in.data
        } .otherwise {
            reluRes.get := 0.S
        }
    }
    
    val inters = (idRes :: reluRes :: Nil) filter ( _.isDefined ) map ( _.get )
    io.out := PriorityMux(io.in.fSel, inters)
}

defined function checkparamsNLU
defined class NonlinearUnit

## Control

Control is implemented via a State Machine (Moore) and Decoder. The state machine must have its states and state transitions configured. The Decoder must have its input-output map configured.

### State Machine

The state machine takes a map as a parameter and constructs the appropriate hardware implementation. The State Machine is configured via the "nextState" parameter: a map from the current state and current input to the next state.

In [10]:
class StateMachineConfig(
        val numStates: Int, 
        val numCtrlSigs: Int, 
        val stateMap: (UInt, UInt, StateMachineConfig) => UInt) {
    
    val stateWidth = log2Up(numStates)
    val ctrlWidth = log2Up(numCtrlSigs)
}

class StateMachine(c: StateMachineConfig) extends Module {
    
    val stateWidth: Int = log2Up(c.numStates)
    
    val io = IO(new Bundle {
        val control = Input (UInt(c.ctrlWidth.W ))
        val out     = Output(UInt(c.stateWidth.W))
    })
    
    val register = RegInit(0.U(c.stateWidth.W))
    register := c.stateMap(register, io.control, c)
    io.out := register
}

defined class StateMachine

### Decoder

The decoder acts as is typical in processors: it converts the state information into all the control signals associated with the appropriate state. It must be configured via the "decode" parameter, a map from the state and control signal name to the desired value.

In [0]:
class DecoderConfig(
        val weightPRFConfig: PRFConfig,
        val actvtnPRFConfig: PRFConfig,
        val intrnlPRFConfig: PRFConfig,
        val ipuConfig: IPUConfig,
        val aluConfig: ALUConfig,
        val nluConfig: NLUConfig,
        val smConfig: StateMachineConfig,
        val decodeWeightPRF: (UInt, PRFConfig) => Data,
        val decodeActvtnPRF: (UInt, PRFConfig) => Data,
        val decodeIntrnlPRF: (UInt, PRFConfig) => Data,
        val decodeIPU: (UInt, IPUConfig) => Data,
        val decodeALU: (UInt, ALUConfig) => Data,
        val decodeNLU: (UInt, NLUConfig) => Data)

class MemoryControl(c: DecoderConfig) extends Bundle {
    
    override def cloneType = (new MemoryControl(c)).asInstanceOf[this.type]
    
    val weightPRF = Output(new PRFControl(c.weightPRFConfig))
    val actvtnPRF = Output(new PRFControl(c.actvtnPRFConfig))
    val intrnlPRF = Output(new PRFControl(c.intrnlPRFConfig))
}

class ProcessControl(c: DecoderConfig) extends Bundle {
    
    override def cloneType = (new ProcessControl(c)).asInstanceOf[this.type]
    
    val aluFSel = Output(Vec(c.aluConfig.numFuncs, Bool()))
    val nluFSel = Output(Vec(c.nluConfig.numFuncs, Bool()))
    
    val ports = c.weightPRFConfig.ports
    val bpFirm = c.ipuConfig.bpFirm
    
    val ipuBpSel = if (bpFirm) Some(Output(Vec(ports, Bool()))) else None
}

class Decoder(c: DecoderConfig) extends Module {
    
    val io = IO(new Bundle {
        val state = Input(UInt(c.smConfig.stateWidth.W))
        val mem = Output(new MemoryControl(c))
        val proc = Output(new ProcessControl(c))
    })
    
    io.mem.weightPRF <> c.decodeWeightPRF(io.state, c.weightPRFConfig)
    io.mem.actvtnPRF <> c.decodeActvtnPRF(io.state, c.actvtnPRFConfig)
    io.mem.intrnlPRF <> c.decodeIntrnlPRF(io.state, c.intrnlPRFConfig)
    
    if (c.ipuConfig.bpFirm) { 
        io.proc.ipuBpSel.get := c.decodeIPU(io.state, c.ipuConfig)
    }
    
    io.proc.aluFSel := c.decodeALU(io.state, c.aluConfig)
    io.proc.nluFSel := c.decodeNLU(io.state, c.nluConfig)
}

cmd0.sc:2: not found: type PRFConfig
        val weightPRFConfig: PRFConfig,
                             ^cmd0.sc:3: not found: type PRFConfig
        val actvtnPRFConfig: PRFConfig,
                             ^cmd0.sc:4: not found: type PRFConfig
        val intrnlPRFConfig: PRFConfig,
                             ^cmd0.sc:5: not found: type IPUConfig
        val ipuConfig: IPUConfig,
                       ^cmd0.sc:6: not found: type ALUConfig
        val aluConfig: ALUConfig,
                       ^cmd0.sc:7: not found: type NLUConfig
        val nluConfig: NLUConfig,
                       ^cmd0.sc:8: not found: type StateMachineConfig
        val smConfig: StateMachineConfig,
                      ^cmd0.sc:9: not found: type UInt
        val decodeWeightPRF: (UInt, PRFConfig) => Data,
                              ^cmd0.sc:9: not found: type PRFConfig
        val decodeWeightPRF: (UInt, PRFConfig) => Data,
                                    ^cmd0.sc:9: not found: type Data
  

: 

## Putting it all Together

In [18]:
class nPE(stateMap: Map[(UInt, UInt), UInt], extrnl_ctrl_width: Int, // State Machine
          decode: (UInt, String) => Data, RFports: Int, weightRFBP: String, actvtnRFBP: String, datawidth: Int, addrwidth: Int,
          aluFuncs: List[String], nluFuncs: List[String], intrnlRFBP: String
         ) extends Module {
    
    val io = IO(new Bundle {
        val extrnl_ctrl   = Input (SInt(extrnl_ctrl_width.W))
        val weightRF_in   = Input (Vec(RFports, SInt(datawidth.W)))
        val actvtnRF_in   = Input (Vec(RFports, SInt(datawidth.W)))
        val intrnlRF_in   = Input (SInt(datawidth.W))
        val weightRF_2NoC = Output(Vec(RFports, SInt(datawidth.W)))
        val actvtnRF_2NoC = Output(Vec(RFports, SInt(datawidth.W)))
        val intrnlRF_2NoC = Output(SInt(datawidth.W))
        val output        = Output(SInt(datawidth.W))
    })
    
    val stateMachine = new StateMachine(stateMap, extrnl_ctrl_width)
    stateMachine.io.control := io.extrnl_ctrl
    
    val decoder = new Decoder(decode, log2Up(stateMap.size), 
                              RFports, datawidth, addrwidth, aluFuncs, nluFuncs)
    decoder.io.state := stateMachine.io.state
    
    
    // Weight RF
    val weightRF = new pRF(RFports, weightRFBP, datawidth, addrwidth)
    
    // Mandatory Control
    weightRF.io.write_en    := decoder.io.weightRF_wen
    weightRF.io.read_en     := decoder.io.weightRF_ren
    weightRF.io.waddr       := decoder.io.weightRF_waddr
    weightRF.io.raddr_int   := decoder.io.weightRF_raddr_int
    weightRF.io.raddr_ext   := decoder.io.weightRF_raddr_ext
    
    // Optional Control
    if ( weightRF.io.bp_slct.isDefined ) { weightRF.io.bp_slct.get := decoder.io.weightRF_bp_slct_get }
    
    // Mandatory Outputs
    weightRF.io.wdata := io.weightRF_in
    
    // Optional Outputs
    io.weightRF_2NoC  := weightRF.io.rdata_ext
    
    // Activation RF
    val actvtnRF = new pRF(RFports, actvtnRFBP, datawidth, addrwidth)
    
    // Mandatory Control
    actvtnRF.io.write_en    := decoder.io.actvtnRF_wen
    actvtnRF.io.read_en     := decoder.io.actvtnRF_ren
    actvtnRF.io.waddr       := decoder.io.actvtnRF_waddr
    actvtnRF.io.raddr_int   := decoder.io.actvtnRF_raddr_int
    actvtnRF.io.raddr_ext   := decoder.io.actvtnRF_raddr_ext
    
    // Optional Control
    if ( actvtnRF.io.bp_slct.isDefined ) { actvtnRF.io.bp_slct.get := decoder.io.actvtnRF_bp_slct_get }
    
    // Mandatory Outputs
    actvtnRF.io.wdata := io.weightRF_in
    
    // Optional Outputs
    io.actvtnRF_2NoC     := actvtnRF.io.rdata_ext
       
    val ipuBP = if(aluFuncs.contains("Add") || aluFuncs.contains("Max")) "Firm" else "None" 
    val ipu   = new IPU(RFports, ipuBP, datawidth)
    if (ipu.io.sel.isDefined) { ipu.io.sel.get := decoder.io.ipu_sel_get }
    ipu.io.in1 := weightRF.io.rdata_int
    ipu.io.in2 := actvtnRF.io.rdata_int
    
    val alu = new ALU(aluFuncs, datawidth)
    alu.io.func_slct := decoder.io.alu_func_slct
    alu.io.innr_prod := ipu.io.out
    if(alu.io.weight_bp.isDefined) alu.io.weight_bp.get := ipu.io.bp1.get
    if(alu.io.actvtn_bp.isDefined) alu.io.actvtn_bp.get := ipu.io.bp2.get
    
    val intrnlRF = new pRF(1, intrnlRFBP, datawidth, addrwidth)
    intrnlRF.io.write_en  := decoder.io.intrnlRF_write_en
    intrnlRF.io.read_en   := decoder.io.intrnlRF_read_en
    intrnlRF.io.waddr     := decoder.io.intrnlRF_waddr
    intrnlRF.io.raddr_int := decoder.io.intrnlRF_raddr_int
    intrnlRF.io.raddr_ext := decoder.io.intrnlRF_raddr_ext
    if (intrnlRF.io.bp_slct.isDefined) { intrnlRF.io.bp_slct.get := decoder.io.intrnlRF_bp_slct_get }
    intrnlRF.io.wdata := Mux(decoder.io.intrnlRF_wdata_slct, alu.io.output, io.intrnlRF_in)
    io.intrnlRF_2NoC := intrnlRF.io.rdata_ext
    if(alu.io.rf_feedbk.isDefined) alu.io.rf_feedbk.get := intrnlRF.io.rdata_int
    
    val nlu = new NonlinearUnit(nluFuncs, datawidth)
    nlu.io.fslct := decoder.io.nlu_func_slct
    nlu.io.input     := intrnlRF.io.rdata_int
    io.output        := nlu.io.outpt
    
    // Woot woot
}

defined class nPE

## Future Plans

Basing the nPE off bypasses (i.e. subtractive modifications to a preconfigured topology) is limiting. A constructive approach would be much more general. This should be accomplished through a DSL and much more fully utilizing the functional programming capabilities of Scala.